In [65]:
# things we need for NLP
import nltk
import pickle
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
import json
import random
import numpy as np

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


# restore all of our data structures
data = pickle.load( open( "training_data.parquet", "rb" ) )
# words = data['words']
# classes = data['classes']
# train_x = data['train_x']
# train_y = data['train_y']
# print(classes)
# print(train_x[0])
# print(train_y[0])


words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [66]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
# print (len(documents), "documents")
# print (len(classes), "classes", classes)
# print (len(words), "unique lemmatized words", words)

In [67]:
import cloudpickle as cp
dataprep_pipeline = cp.load(open('01_chatbot_data_preparation.sav', 'rb'))

In [68]:
training = dataprep_pipeline.transform(documents)
print(training[0])
train_x = list(training[:,0])
train_y = list(training[:,1])
print(type(training[:,0]))

[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
 list([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])]
<class 'numpy.ndarray'>


### Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons equal to number of intents to predict output intent with softmax

In [69]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
dropout=0.5
optimizer=sgd

def build_model(optimizer='adam', dropout=0.5):
    model = Sequential()
    model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(len(train_y[0]), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


### Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model

In [74]:
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn= build_classifier ,optimizer=sgd, epochs=200, batch_size=5, verbose=1)
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)


# pipeline_keras = Pipeline([('clf', classifier)])

# hist = pipeline_keras.fit(np.array(train_x), np.array(train_y))

# param_grid = {
#     'clf__optimizer':[sgd,'adam'],
#     'clf__epochs':[100],
#     'clf__dropout':[0.5,0.4],
# }
# grid_search = GridSearchCV(estimator = pipeline,
#                        param_grid = param_grid,
#                        scoring = 'accuracy',
#                        cv=2)


# grid_search.fit(np.array(train_x), np.array(train_y))

# print(train_x[:1])
# classifier.model.predict(np.array(train_x[:1]))

Epoch 1/200
20/20 [==============================] - 0s 1ms/step - loss: 2.4719 - accuracy: 0.1237
Epoch 2/200
20/20 [==============================] - 0s 2ms/step - loss: 2.3275 - accuracy: 0.1959
Epoch 3/200
20/20 [==============================] - 0s 2ms/step - loss: 2.1775 - accuracy: 0.2990
Epoch 4/200
20/20 [==============================] - 0s 2ms/step - loss: 2.0648 - accuracy: 0.2990
Epoch 5/200
20/20 [==============================] - 0s 1ms/step - loss: 1.9048 - accuracy: 0.3814
Epoch 6/200
20/20 [==============================] - 0s 2ms/step - loss: 1.7652 - accuracy: 0.4124
Epoch 7/200
20/20 [==============================] - 0s 1ms/step - loss: 1.6530 - accuracy: 0.4124
Epoch 8/200
20/20 [==============================] - 0s 1ms/step - loss: 1.4431 - accuracy: 0.6186
Epoch 9/200
20/20 [==============================] - 0s 2ms/step - loss: 1.4549 - accuracy: 0.5670
Epoch 10/200
20/20 [==============================] - 0s 2ms/step - loss: 1.1738 - accuracy: 0.6598
Epoch 11/

20/20 [==============================] - 0s 2ms/step - loss: 0.0543 - accuracy: 0.9794
Epoch 83/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0554 - accuracy: 0.9897
Epoch 84/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0405 - accuracy: 0.9897
Epoch 85/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0756 - accuracy: 0.9794
Epoch 86/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0672 - accuracy: 0.9794
Epoch 87/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0421 - accuracy: 0.9794
Epoch 88/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0466 - accuracy: 0.9897
Epoch 89/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0303 - accuracy: 0.9897
Epoch 90/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0307 - accuracy: 0.9897
Epoch 91/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0460 - accuracy: 0.9897
Epoch 92/200


Epoch 163/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0176 - accuracy: 1.0000
Epoch 164/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 165/200
20/20 [==============================] - ETA: 0s - loss: 0.0013 - accuracy: 1.00 - 0s 1ms/step - loss: 0.0273 - accuracy: 0.9897
Epoch 166/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0106 - accuracy: 1.0000
Epoch 167/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0122 - accuracy: 1.0000
Epoch 168/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0245 - accuracy: 1.0000
Epoch 169/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0250 - accuracy: 0.9897
Epoch 170/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0071 - accuracy: 1.0000
Epoch 171/200
20/20 [==============================] - 0s 1ms/step - loss: 0.0143 - accuracy: 1.0000
Epoch 172/200
20/20 [============================

### saving the model 

In [79]:
import os
import cloudpickle as cp

default_filename = "02_chatbot_model.sav"
filename = os.getenv("S2I_PIPELINE_STAGE_SAVE_FILE", default_filename)
print("saving model")
# cp.dump(model.model, open(default_filename, "wb"))
# cp.dump(model.model, open(filename, "wb"))
model.model.save(default_filename)
model.model.save(filename)
model.model.save('chatbot_model.h5', hist)
print("model created")

saving model
INFO:tensorflow:Assets written to: 02_chatbot_model.sav/assets
INFO:tensorflow:Assets written to: 02_chatbot_model.sav/assets
model created
